In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transform)

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])

batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

def train(model, epochs=5):
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    for epoch in range(epochs):
        running_loss = 0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch [{epoch+1}] Loss: {running_loss:.4f}")

def evaluate(model):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

resnet50 = models.resnet50(pretrained=True)
resnet50.fc = nn.Linear(resnet50.fc.in_features, 10)
resnet50.to(device)

print("\n Training ResNet-50...")
train(resnet50, epochs=5)
resnet50_acc = evaluate(resnet50)
print(f" ResNet-50 Accuracy on CIFAR-10: {resnet50_acc:.2f}%")

resnet152 = models.resnet152(pretrained=True)
resnet152.fc = nn.Linear(resnet152.fc.in_features, 10)
resnet152.to(device)

print("\n Training ResNet-152...")
train(resnet152, epochs=5)
resnet152_acc = evaluate(resnet152)
print(f" ResNet-152 Accuracy on CIFAR-10: {resnet152_acc:.2f}%")


100%|██████████| 170M/170M [00:06<00:00, 26.6MB/s]
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 182MB/s]



 Training ResNet-50...


Epoch 1: 100%|██████████| 625/625 [07:37<00:00,  1.37it/s]


Epoch [1] Loss: 201.1535


Epoch 2: 100%|██████████| 625/625 [07:37<00:00,  1.36it/s]


...

Epoch [3] Loss: 41.4538


Epoch 4: 100%|██████████| 625/625 [16:49<00:00,  1.62s/it]


Epoch [4] Loss: 41.0101


Epoch 5: 100%|██████████| 625/625 [16:50<00:00,  1.62s/it]


Epoch [5] Loss: 30.4440
 ResNet-152 Accuracy on CIFAR-10: 94.59%


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve
from sklearn.preprocessing import label_binarize

def evaluate_with_preds(model, loader):
    model.eval()
    y_true, y_pred, y_prob = [], [], []
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1).cpu().numpy()
            preds = np.argmax(probs, axis=1)

            y_true.extend(labels.numpy())
            y_pred.extend(preds)
            y_prob.extend(probs)
    return np.array(y_true), np.array(y_pred), np.array(y_prob)

def plot_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    plt.savefig("resnet_confusion_matrix.png")
    plt.close()

def plot_roc_auc(y_true, y_prob, class_names):
    y_true_bin = label_binarize(y_true, classes=np.arange(len(class_names)))
    plt.figure(figsize=(10, 8))
    for i in range(len(class_names)):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_prob[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"{class_names[i]} (AUC={roc_auc:.2f})")
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title("ResNet ROC-AUC Curve (One-vs-All)")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("resnet_roc_auc_curve.png")
    plt.close()

def plot_precision_recall(y_true, y_prob, class_names):
    y_true_bin = label_binarize(y_true, classes=np.arange(len(class_names)))
    plt.figure(figsize=(10, 8))
    for i in range(len(class_names)):
        precision, recall, _ = precision_recall_curve(y_true_bin[:, i], y_prob[:, i])
        plt.plot(recall, precision, label=class_names[i])
    plt.title("ResNet Precision-Recall Curve")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("resnet_precision_recall_curve.png")
    plt.close()


In [3]:

y_true_resnet, y_pred_resnet, y_prob_resnet = evaluate_with_preds(resnet50, test_loader)
class_names = test_dataset.classes

plot_confusion_matrix(y_true_resnet, y_pred_resnet, class_names)
plot_roc_auc(y_true_resnet, y_prob_resnet, class_names)
plot_precision_recall(y_true_resnet, y_prob_resnet, class_names)

print(" ResNet plots saved: resnet_confusion_matrix.png, resnet_roc_auc_curve.png, resnet_precision_recall_curve.png")


✅ ResNet plots saved: resnet_confusion_matrix.png, resnet_roc_auc_curve.png, resnet_precision_recall_curve.png


In [5]:
from google.colab import files

files.download("resnet_confusion_matrix.png")
files.download("resnet_roc_auc_curve.png")
files.download("resnet_precision_recall_curve.png")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>